# **YOLO**

## **IMPORTS**

In [ ]:
!pip install ultralytics --quiet
from ultralytics import YOLO

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="FkIXdywiy0LIlW4hIKko")
project = rf.workspace("tfg-s3wgw").project("orange-detector-v2")
version = project.version(1)
dataset = version.download("yolov11")

## **TRAINING**

In [ ]:
dataset_path = "./Orange-Detector-V2-1/"
dataset_name = "Orange-Detector-V2-1"

!cat {dataset_path}/data.yaml # OPCIONAL: VERIFICAR DATASET

In [ ]:
'''
import os
import random
import shutil

# 1 DATASET
# CREAMOS LAS CARPETAS 'VAL' Y 'TEST' A PARTIR DE 'TRAIN'

# Porcentaje de validación y test
val_ratio = 0.15
test_ratio = 0.15

random.seed(33)

# Crear carpetas de destino
for split in ['valid', 'test']:
    for sub in ['images', 'labels']:
        os.makedirs(os.path.join(dataset_path, split, sub), exist_ok=True)

# Listar imágenes de entrenamiento
train_img_dir = os.path.join(dataset_path, "train", "images")
train_lbl_dir = os.path.join(dataset_path, "train", "labels")
images = [f for f in os.listdir(train_img_dir) if f.endswith(".jpg")]

random.shuffle(images)
n = len(images)
val_split = int(n * val_ratio)
test_split = int(n * test_ratio)

val_imgs = images[:val_split]
test_imgs = images[val_split:val_split + test_split]

# Función para mover imagen y su etiqueta
def mover(im_list, destino):
    for img_name in im_list:
        lbl_name = img_name.replace('.jpg', '.txt')
        shutil.move(os.path.join(train_img_dir, img_name),
                    os.path.join(dataset_path, destino, "images", img_name))
        shutil.move(os.path.join(train_lbl_dir, lbl_name),
                    os.path.join(dataset_path, destino, "labels", lbl_name))

# Mover archivos
mover(val_imgs, "valid")
mover(test_imgs, "test")

print(f"✅ División realizada: {len(val_imgs)} valid | {len(test_imgs)} test | {n - val_split - test_split} train")
'''

In [ ]:
'''
import os
import random
import shutil

# 2 DATASETS (CITDET)

# Porcentaje de validación y test
val_ratio = 0.15
test_ratio = 0.15

random.seed(33)

# Crear carpetas de destino (validación y test)
for split in ['valid', 'test']:
    for sub in ['images', 'labels']:
        os.makedirs(os.path.join(dataset_path, split, sub), exist_ok=True)

# Listar imágenes de tu dataset (sólo imágenes que comienzan con 'arbol')
train_img_dir = os.path.join(dataset_path, "train", "images")
train_lbl_dir = os.path.join(dataset_path, "train", "labels")
images = [f for f in os.listdir(train_img_dir) if f.endswith(".jpg") and f.startswith("arbol")]
imagesAll = [f for f in os.listdir(train_img_dir) if f.endswith(".jpg")]

random.shuffle(images)
n = len(imagesAll)
val_split = int(n * val_ratio)
test_split = int(n * test_ratio)

# Dividir imágenes en validación y test
val_imgs = images[:val_split]
test_imgs = images[val_split:val_split + test_split]

# Función para mover imagen y su etiqueta
def mover(im_list, destino):
    for img_name in im_list:
        lbl_name = img_name.replace('.jpg', '.txt')  # Asumiendo que las etiquetas son .txt
        shutil.move(os.path.join(train_img_dir, img_name),
                    os.path.join(dataset_path, destino, "images", img_name))
        shutil.move(os.path.join(train_lbl_dir, lbl_name),
                    os.path.join(dataset_path, destino, "labels", lbl_name))

# Mover archivos
mover(val_imgs, "valid")
mover(test_imgs, "test")

print(f"✅ División realizada: {len(val_imgs)} valid | {len(test_imgs)} test | {n - val_split - test_split} train")
'''

In [ ]:
# ENTRENAR MODELO

modelo = "yolo11l"

model = YOLO(f"{modelo}.pt")

model.train(
    data=f"/home/quirogaalu/TFG/notebooks/{dataset_name}/data.yaml",
    epochs=500, #300
    imgsz=640,
    batch=8,
    patience=50, #30
    device=0,
    name=f"{modelo}_naranjas",
    project='./runs/detect',
    exist_ok=True,
    save=True,
    plots=True,

    hsv_h=0.05, # 0.02 a 0.1
    hsv_s=0.4, # 0.3 a 0.5
    hsv_v=0.4, # 0.3 a 0.5
    degrees=0.1,
    scale=0.1, # 0.1 a 0.2
    shear=2.5,
    perspective=0.0001,
    fliplr=0.5,

    # DESACTIVADOS: quiero que el modelo se fije en el centro
    translate=0.0,
    mosaic=0.0,
    mixup=0.0,
    cutmix=0.0
)

## **TEST**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# METRICAS DE ENTRENAMIENTO

df = pd.read_csv(f'./runs/detect/{modelo}_naranjas/results.csv')

# Imprimir las columnas disponibles para inspección
print(df.columns)

# Acceder a las columnas por su nombre real
plt.plot(df['epoch'], df['metrics/precision(B)'], label='Precisión de Entrenamiento')
plt.plot(df['epoch'], df['metrics/recall(B)'], label='Recall de Entrenamiento')

# Graficar la pérdida
plt.plot(df['epoch'], df['train/box_loss'], label='Pérdida de Entrenamiento (Box)')
plt.plot(df['epoch'], df['val/box_loss'], label='Pérdida de Validación (Box)')

# Agregar detalles
plt.xlabel("Época")
plt.ylabel("Valor")
plt.title("Evolución del Entrenamiento")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from ultralytics import RTDETR

%matplotlib inline

# EVALUAR EL MODELO

model = RTDETR("/home/quirogaalu/TFG/models/rt-detr-l.pt")
#model = YOLO("/home/quirogaalu/TFG/models/yolov11l_citdet.pt")

metrics = model.val(data=f"/home/quirogaalu/TFG/notebooks/{dataset_name}/data.yaml", imgsz=1024)
#metrics = model.val(data=f"/home/quirogaalu/TFG/notebooks/{dataset_name}/data.yaml", conf=0.001, iou=0.5, imgsz=1024)

precision = metrics.box.p.mean()
recall = metrics.box.r.mean()
map50 = metrics.box.map50
map5095 = metrics.box.map


# Mostrar numéricamente
print(f"\n📊 MÉTRICAS DE DESEMPEÑO:")
print(f"Precision:   {precision:.3f}")
print(f"Recall:      {recall:.3f}")
print(f"mAP50:       {map50:.3f}")
print(f"mAP50-95:    {map5095:.3f}\n")

# Gráfica de barras horizontal
labels = ['Precision', 'Recall', 'mAP50', 'mAP50-95']
values = [precision, recall, map50, map5095]

plt.figure(figsize=(7, 4))
plt.barh(labels, values, color=['#4caf50', '#2196f3', '#ff9800', '#9c27b0'])
plt.xlim(0, 1)
plt.xlabel('Valor')
plt.title('Métricas de evaluación')
for i, v in enumerate(values):
    plt.text(v + 0.02, i, f"{v:.2f}", va='center')
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# HACER PREDICCIONES SOBRE TEST

results = model.predict(source=f"{dataset_path}/test/images", save=True, imgsz=1024)

In [ ]:
!zip -r ./yolo_results.zip ./runs/detect

# **RT-DETR**

## **IMPORTS**

In [ ]:
!pip install ultralytics --quiet
from ultralytics import YOLO

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="FkIXdywiy0LIlW4hIKko")
project = rf.workspace("tfg-s3wgw").project("orange-detector-v2")
version = project.version(5)
dataset = version.download("yolov11")

## **TRAINING**

In [ ]:
dataset_path = "./Orange-Detector-V2-5/"
dataset_name = "Orange-Detector-V2-5"

!cat {dataset_path}/data.yaml # OPCIONAL: VERIFICAR DATASET

In [ ]:
'''
import os
import random
import shutil

# 1 DATASET
# CREAMOS LAS CARPETAS 'VAL' Y 'TEST' A PARTIR DE 'TRAIN'

# Porcentaje de validación y test
val_ratio = 0.15
test_ratio = 0.15

random.seed(33)

# Crear carpetas de destino
for split in ['valid', 'test']:
    for sub in ['images', 'labels']:
        os.makedirs(os.path.join(dataset_path, split, sub), exist_ok=True)

# Listar imágenes de entrenamiento
train_img_dir = os.path.join(dataset_path, "train", "images")
train_lbl_dir = os.path.join(dataset_path, "train", "labels")
images = [f for f in os.listdir(train_img_dir) if f.endswith(".jpg")]

random.shuffle(images)
n = len(images)
val_split = int(n * val_ratio)
test_split = int(n * test_ratio)

val_imgs = images[:val_split]
test_imgs = images[val_split:val_split + test_split]

# Función para mover imagen y su etiqueta
def mover(im_list, destino):
    for img_name in im_list:
        lbl_name = img_name.replace('.jpg', '.txt')
        shutil.move(os.path.join(train_img_dir, img_name),
                    os.path.join(dataset_path, destino, "images", img_name))
        shutil.move(os.path.join(train_lbl_dir, lbl_name),
                    os.path.join(dataset_path, destino, "labels", lbl_name))

# Mover archivos
mover(val_imgs, "valid")
mover(test_imgs, "test")

print(f"✅ División realizada: {len(val_imgs)} valid | {len(test_imgs)} test | {n - val_split - test_split} train")
'''

In [ ]:
# ENTRENAR MODELO

from ultralytics import RTDETR

model = RTDETR("rtdetr-l.pt")

model.train(
    data=f"/home/quirogaalu/TFG/notebooks/{dataset_name}/data.yaml",
    epochs=500, #300
    imgsz=1024,
    batch=8,
    patience=50, #30
    device=0,
    name="RTDETR_naranjas",
    project='./runs/detect',
    exist_ok=True,
    save=True,
    plots=True,

    hsv_h=0.05, # 0.02 a 0.1
    hsv_s=0.4, # 0.3 a 0.5v
    hsv_v=0.4, # 0.3 a 0.5
    degrees=0.1,
    scale=0.1, # 0.1 a 0.2
    shear=2.5,
    perspective=0.0001,
    fliplr=0.5,
    flipud=0.0,

    # DESACTIVADOS: quiero que el modelo se fije en el centro
    translate=0.0,
    mosaic=0.0,
    mixup=0.0,
    cutmix=0.0
)

## **TEST**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Cargar CSV con resultados de entrenamiento
df = pd.read_csv('./runs/detect/RTDETR_naranjas/results.csv')

# Graficar métricas principales
plt.figure(figsize=(10, 6))

plt.plot(df['epoch'], df['metrics/precision(B)'], label='Precisión')
plt.plot(df['epoch'], df['metrics/recall(B)'], label='Recall')
plt.plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@0.5')

plt.xlabel("Época")
plt.ylabel("Valor")
plt.title("Evolución de Precisión, Recall y mAP durante el Entrenamiento")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Graficar pérdidas
plt.figure(figsize=(10, 6))

plt.plot(df['epoch'], df['train/giou_loss'], label='GIoU Loss (Train)')
plt.plot(df['epoch'], df['val/giou_loss'], label='GIoU Loss (Val)')
plt.plot(df['epoch'], df['train/cls_loss'], label='Class Loss (Train)')
plt.plot(df['epoch'], df['val/cls_loss'], label='Class Loss (Val)')
plt.plot(df['epoch'], df['train/l1_loss'], label='L1 Loss (Train)')
plt.plot(df['epoch'], df['val/l1_loss'], label='L1 Loss (Val)')

plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.title("Evolución de las Pérdidas durante el Entrenamiento")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# EVALUAR EL MODELO

#metrics = model.val(conf=0.4, iou=0.5)
metrics = model.val()

precision = metrics.box.p.mean()
recall = metrics.box.r.mean()
map50 = metrics.box.map50
map5095 = metrics.box.map


# Mostrar numéricamente
print(f"\n📊 MÉTRICAS DE DESEMPEÑO:")
print(f"Precision:   {precision:.3f}")
print(f"Recall:      {recall:.3f}")
print(f"mAP50:       {map50:.3f}")
print(f"mAP50-95:    {map5095:.3f}\n")

# Gráfica de barras horizontal
labels = ['Precision', 'Recall', 'mAP50', 'mAP50-95']
values = [precision, recall, map50, map5095]

plt.figure(figsize=(7, 4))
plt.barh(labels, values, color=['#4caf50', '#2196f3', '#ff9800', '#9c27b0'])
plt.xlim(0, 1)
plt.xlabel('Valor')
plt.title('Métricas de evaluación')
for i, v in enumerate(values):
    plt.text(v + 0.02, i, f"{v:.2f}", va='center')
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# HACER PREDICCIONES SOBRE TEST

results = model.predict(source=f"{dataset_path}/test/images", save=True, imgsz=1024)

In [ ]:
!zip -r ./detr_results.zip ./runs/detect

# **FASTER R-CNN**

## **IMPORTS**

In [ ]:
import os
import torch, detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.logger import setup_logger

import numpy as np
import cv2
import random

setup_logger()

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="FkIXdywiy0LIlW4hIKko")
project = rf.workspace("tfg-s3wgw").project("orange-detector-v2")
version = project.version(1)
dataset = version.download("coco")

## **PREPROCESADO**

In [ ]:
#!rm -rf /content/Orange-Detector-5/

In [ ]:
#!mv ./Orange-Detector-5 ../datasets/roboflow/

In [ ]:
from detectron2.data.datasets import register_coco_instances

dataset_path = "./Orange-Detector-V2-1/"

register_coco_instances("oranges_train", {}, f"{dataset_path}train/_annotations.coco.json", f"{dataset_path}train/")

register_coco_instances("oranges_val", {}, f"{dataset_path}valid/_annotations.coco.json", f"{dataset_path}valid/")

register_coco_instances("oranges_test", {}, f"{dataset_path}test/_annotations.coco.json", f"{dataset_path}test/")

In [ ]:
import matplotlib.pyplot as plt
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

my_dataset_train_metadata = MetadataCatalog.get("oranges_train")
dataset_dicts = DatasetCatalog.get("oranges_train")

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image())
    plt.axis('off')
    plt.show()

## **TRAINING**

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
import detectron2.data.transforms as T
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, MetadataCatalog, build_detection_train_loader
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("oranges_train",)
cfg.DATASETS.TEST = ("oranges_val",)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 8 # Batch Size
cfg.SOLVER.BASE_LR = 0.01 # Learning Rate
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 2000  # Epochs
cfg.SOLVER.STEPS = (1500, 1750)  # learning rate decay schedule
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 # The "RoIHead batch size" (512 = default)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.TEST.EVAL_PERIOD = 250  # evalúa cada 250 iteraciones

dataloader = build_detection_train_loader(cfg,
   mapper=DatasetMapper(cfg, is_train=True, augmentations=[
      T.RandomApply(T.RandomFlip(horizontal=True, vertical=False), prob=0.5),
      T.RandomApply(T.RandomRotation(angle=[-10, 10], expand=False, center=None, sample_style='range', interp=None), prob=0.5),
      T.RandomApply(T.RandomContrast(intensity_min=0.9, intensity_max=1.1), prob=0.5),
      T.RandomApply(T.RandomSaturation(intensity_min=0.6, intensity_max=1.4), prob=0.5),
      T.RandomApply(T.RandomBrightness(intensity_min=0.9, intensity_max=1.1), prob=0.5),
      T.RandomApply(T.RandomLighting(scale=0.1), prob=0.5)
   ]))

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

## **TEST**

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor

# SI SE QUIERE CARGAR DIRECTAMENTE EL MODELO

cfg = get_cfg()
cfg.merge_from_file("/home/quirogaalu/TFG/config_faster_rcnn_citdet.yaml")

#cfg.DATASETS.TRAIN = ("oranges_train",)
#cfg.DATASETS.TEST = ("oranges_test",)

cfg.MODEL.WEIGHTS = "/home/quirogaalu/TFG/models/faster_rcnn_citdet.pth"

#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.001
#cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)

evaluator = COCOEvaluator("oranges_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "oranges_test")
results = inference_on_dataset(predictor.model, val_loader, evaluator)

print("📊 Resultados de evaluación:")
print(results)

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor

# SI SE QUIERE CARGAR DIRECTAMENTE EL MODELO
#cfg = get_cfg()
#cfg.merge_from_file("ruta_a_config.yaml")

# After training, load the model weights:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Umbral de confianza

# Create a COCO evaluator and data loader:
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("oranges_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "oranges_test")

# Run inference and evaluation:
results = inference_on_dataset(trainer.model, val_loader, evaluator)
print("📊 Resultados de evaluación:")
print(results)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("oranges_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # umbral de confianza para inferencia
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("oranges_test")

In [ ]:
with open("./output/config.yaml", "w") as f:
    f.write(cfg.dump())

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer

# VER PREDICCIONES SOBRE LAS IMÁGENES
output_dir = './output/'  # Define el directorio donde quieres guardar las imágenes
os.makedirs(output_dir, exist_ok=True)

for imageName in glob.glob(f'{dataset_path}/test/*jpg'):
    im = cv2.imread(imageName)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=test_metadata, scale=0.8)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Guardar la imagen procesada
    output_image_path = f"{output_dir}/{imageName.split('/')[-1]}"  # Cambia el nombre del archivo si es necesario
    cv2.imwrite(output_image_path, out.get_image()[:, :, ::-1])  # Guarda la imagen en BGR (OpenCV usa BGR, no RGB)

    # Visualización con matplotlib (opcional)
    plt.figure(figsize=(12, 8))  # Tamaño grande en pulgadas (ancho x alto)
    plt.imshow(out.get_image())
    plt.title(imageName.split("/")[-1])
    plt.axis('off')
    plt.show()

In [ ]:
!zip -r ./fasterrcnn_results.zip ./output/